In [105]:
from bs4 import BeautifulSoup
import requests
import re

In [111]:
def extract_book_info(book_id):
    dic = {}
    book_url = 'https://www.moscowbooks.ru' + str(book_id)
    for attempt in range(attempt_count):
        html_1 = requests.get(book_url)
        if html_1.status_code == 200:
            break
        if html_1.status_code != 200 and attempt == attempt_count - 1:
            return dic
        
    soup = BeautifulSoup(html_1.text, 'html.parser')
    
    dic['Код товара'] = book_id[1:-1].split('/')[1]
    
    header = soup.find(class_='page-header')
    
    y =  header.find(class_='page-header__author')
    y = y.find_all(class_ = 'author-name')
    dic['Автор'] = ', '.join(k.text.strip() for k in y)
    
    y = header.find(class_ = 'page-header__title')
    dic['Название'] = soup.find('span', class_ = 'link-gray-light').text.strip()
    
    body = soup.find(class_ = 'book__body')
    intro = body.find(class_ = 'book__intro')
    info = body.find(class_ = 'book__info')
    
    y = intro.find(class_ = 'book__cover').find(class_ = 'link_gallery')
    dic['Обложка'] = 'https://www.moscowbooks.ru' + y['href']
    
    y =  body.find(class_='book___rating-stars')
    dic['Рейтинг'] = int(y['data-rate'])
    
    y = intro.find(class_ = 'book__stickers')
    if y:
        gift = y.find(class_='stickers__icon icon-gift')
        gift = [] if not gift else [gift['title']]     
        y =', '.join(item for item in y.text.split('\n') + gift if item)
        dic['Стикеры'] = y
            
    y = intro.find(class_ = 'book__shop-details').find(class_ = 'icon-check')
    dic['Наличие'] = y is not None
    
    y = intro.find(class_ = 'book__buy').find(class_ = 'book__price').text
    dic['Цена'] = ''.join(k for k in y if str.isdigit(k)) + ' руб.'
    
    tmp = info.find_all('dl', class_='book__details-item')
    res = {cur.find('dt', class_='book__details-name').text.strip()[:-1]:\
           cur.find('dt', class_='book__details-value').text.strip() for cur in tmp}
    dic.update(res)
    #сделал код короче, но, к сожалению, только book__description оставить не удалось,
    #так как,например в книге https://www.moscowbooks.ru/book/764719 тэги имеют тип book__description
    #и не удается вывести описание за один info.find, сократив при этом код.
    y =  info.find(class_ = 'book__description collapsed js-book-description').find_all(text = True, recursive = False)
    y = ' '.join(y).strip()
    dic['Описание'] = y
    
    return dic


In [112]:
autors = {
  "Достоевский Ф. М.": 9150,
  "Роллинс Дж.": 59396,
  "Фицджеральд Ф. С.": 28727,
  "Глуховский Д. А.": 53427,
  "Стругацкий А. Н.": 26268,
  "Лукьяненко С. В.": 16626,
  "Фрай М.": 28927,
  "Хантер Э.": 37969,
  "Роулинг Дж. К.": 104832
}
book_ids = []
for k in autors.values():
    link = 'https://www.moscowbooks.ru/catalog/author/' + str(k) +'/'
    next_ = 1
    while (next_ != None):
        html_1 = requests.get(link).text
        soup = BeautifulSoup(html_1, 'html.parser')
        y = soup.find(class_='catalog__list').find_all(class_='book-preview__title-link')
        for j in y:
            book_ids.append(j['href'])  
        next_ = soup.find(class_ = 'pager__text', title = 'следующая страница')
        if (next_ != None):
            link = 'https://www.moscowbooks.ru' + next_['href']                                                     

In [113]:
len(book_ids)

243

In [114]:
from multiprocessing import Pool, Lock, Value
from time import sleep

mutex = Lock()
n_processed = Value('i', 0)
attempt_count = 5

def func_wrapper(book_ids):
    global attempt_count
    res = extract_book_info(book_ids) 
    with mutex:
        # в этом блоке можно безопасно менять общие объекты для процессов
        #что-то вроде MIAD
        if res == {}:
            attempt_count *= 2
        else:
            attempt_count = max(1, attempt_count - 1)
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    res = pool.map(func_wrapper, book_ids)

240 objects are processed...

In [115]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

df = pd.DataFrame(res)
df

,Код товара,Автор,Название,Обложка,Рейтинг,Стикеры,Наличие,Цена,Издательство,Год издания,Место издания,Язык текста,Тип обложки,Формат,Размеры в мм (ДхШхВ),Вес,Страниц,Тираж,Артикул,ISBN,В продаже с,Описание,Возраст,Тиснение,Бумага,Обрез,Иллюстрации,Язык оригинала,Перевод,Футляр,Производитель,Год производства,Место производства,Иллюстраторы
0,1012089,Достоевский Ф. М.,Преступление и наказание,https://www.moscowbooks.ru/image/book/675/w600...,5,,True,136 руб.,Эксмо,2019,Москва,русский,Твердый переплет,84х108 1/32,200x130x28,425 гр.,592,7000 экз.,ITD000000000832998,978-5-699-93016-6,11.10.2019,"""Преступление и наказание"" — высочайший образе...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1004205,Достоевский Ф. М.,Игрок,https://www.moscowbooks.ru/image/book/668/w600...,0,,False,220 руб.,Искателькнига,2015,Москва,русский,Твердый переплет,60х90 1/16,215x145,227 гр.,126,3000 экз.,NaN,978-5-9905833-4-4,27.08.2019,"В романе ""Игрок"" писатель изображает прекрасно...",16 +,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000435,Достоевский Ф. М.,Чужая жена и муж под кроватью,https://www.moscowbooks.ru/image/book/664/w600...,0,,True,144 руб.,АСТ,2019,Москва,русский,Мягкая обложка,76х100 1/32,180x115x27,240 гр.,384,5000 экз.,ASE000000000843846,978-5-17-115705-0,30.07.2019,Очень необычный сборник. Очень непривычный Дос...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,997191,Достоевский Ф. М.,Полное собрание романов в двух томах. В 2 книгах,https://www.moscowbooks.ru/image/book/661/w600...,0,,True,1880 руб.,АЛЬФА-КНИГА,2019,Москва,русский,Твердый переплет,60х90 1/16,215x145x115,2445 гр.,2558,3000 экз.,188881,978-5-9922-0332-5,08.07.2019,"В первый том вошли романы ""Бедные люди"", ""Игро...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,995794,Достоевский Ф. М.,Преступление и наказание,https://www.moscowbooks.ru/image/book/659/w600...,0,NaN,True,12654 руб.,NaN,2019,СПб,русский,Кожаный переплет,NaN,241x175x45,1100 гр.,432,NaN,NaN,978-5-9603-0470-2,27.06.2019,"""Преступление и наказание"" — одно из самых изв...",NaN,Углубленное тиснение (красочное плоскоуглублен...,Офсетная,Рисованный,С иллюстрациями,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,994923,Достоевский Ф. М.,Село Степанчиково и его обитатели. Из записок ...,https://www.moscowbooks.ru/image/book/658/w600...,0,,True,778 руб.,Издательский дом Мещерякова,2019,Москва,русский,Твердый переплет,60х84 1/16,200x145x22,560 гр.,296,1500 экз.,IDM00006739,978-5-00108-509-6,21.06.2019,"В своей повести ""Село Степанчиково и его обита...",10 +,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,992455,Достоевский Ф. М.,Собрание сочинений. В 10 томах,https://www.moscowbooks.ru/image/book/656/w600...,0,Идея подарка,True,6400 руб.,Книжный Клуб Книговек,2019,Москва,русский,Твердый переплет,84х108 1/32,200x130x345,6960 гр.,NaN,NaN,СМ017429,978-5-4224-1537-3,30.05.2019,Федор Михайлович Достоевский (1821—1881) — вел...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,989763,Достоевский Ф. М.,Малая проза. Книга 2. Скверный анекдот. Крокод...,https://www.moscowbooks.ru/image/book/654/w600...,0,NaN,True,4880 руб.,NaN,2019,СПб,русский,Твердый переплет,70х100 1/16,240x170x40,1510 гр.,536,800 экз.,NaN,978-5-93898-696-1,15.05.2019,"Настоящее издание ""малой прозы"" Ф. М. Достоевс...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,988509,Достоевский Ф. М.,Братья Карамазовы,https://www.moscowbooks.ru/image/book/653/w600...,0,NaN,True,6318 руб.,NaN,2019,СПб,русский,Кожаный переплет,60х90 1/16,215x145x45,960 гр.,832,NaN,978-5-389-15519-0,NaN,30.04.2019,"«Братья Карамазовы» (1879–1880), последний, са...",NaN,Углубленное тиснение (красочное плоскоуглублен...,Офсетная,Рисованный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,987119,Достоевский Ф. М.,Преступление и наказание,https://www.moscowbooks.ru/image/book/652/w600...,0,,True,630 руб.,СЗКЭО,2019,СПб,русский,Твердый переплет,NaN,241x175x41,990 гр.,432,3000 экз.,ББ29279,978-5-9603-0470-2,22.04.2019,"""Преступление и наказание"" — одно из самых изв...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [40]:
df = pd.DataFrame(res)
df.sort_values(by=['Код товара'], inplace=True)

with open('homework_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)